In [1]:
!pip install music21 tensorflow numpy


In [5]:
from music21 import converter, instrument, note, chord

def parse_midi(file_path):
    midi = converter.parse(file_path)
    notes = []
    for element in midi.flatten().notes:  # Updated to use .flatten()
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

# Specify the path to your MIDI file
midi_file = "C:/Users/fer_v/OneDrive/Desktop/cello_suite_1.mid"
notes = parse_midi(midi_file)


In [7]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

encoder = LabelEncoder()
encoded_notes = encoder.fit_transform(notes)

sequence_length = 100
network_input = []
network_output = []

for i in range(len(encoded_notes) - sequence_length):
    seq_in = encoded_notes[i:i + sequence_length]
    seq_out = encoded_notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

network_input = np.reshape(network_input, (len(network_input), sequence_length, 1))
network_input = network_input / float(len(set(encoded_notes)))
network_output = np.array(network_output)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation, Input

# Define the model architecture using the Input layer
model = Sequential([
    Input(shape=(network_input.shape[1], network_input.shape[2])),  # Use Input layer to define input shape
    LSTM(512, return_sequences=True),
    Dropout(0.3),
    LSTM(512, return_sequences=True),
    Dropout(0.3),
    LSTM(512),
    Dense(256),
    Activation('relu'),
    Dropout(0.3),
    Dense(len(set(encoded_notes))),
    Activation('softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Train the model
model.fit(network_input, tf.keras.utils.to_categorical(network_output), epochs=100, batch_size=64)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 3.4598
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 3.1003
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 3.0837
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 2.9111
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.9322
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.9785
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 2.9563
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.9137
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.9187
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.9095
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 2.9145
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 2.9009
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.8917
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 2.8956
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 2.8923
Epoch 16/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2

In [24]:
import numpy as np
from music21 import stream, note, chord, instrument

# Example data: Replace 'notes' with your actual data
notes = ["C4", "E4", "G4", "C5", "E5", "G5", "C6", "E6", "G6"]  # Example notes
unique_notes = sorted(set(notes))

# Map integers to notes and notes to integers
int_to_note = {number: note for number, note in enumerate(unique_notes)}
note_to_int = {note: number for number, note in int_to_note.items()}

def generate_music(model, network_input, int_to_note, num_notes=500):
    # Select a random seed
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    # Generate notes
    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(set(note_to_int)))
        
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note.get(index, "C4")  # Default to "C4" if index not found
        prediction_output.append(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    print("Generated Music:", prediction_output)  # Debug statement
    return prediction_output

def create_midi(prediction_output):
    if not prediction_output:
        print("No prediction output to create MIDI.")
        return
    
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        # If the pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # If the pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='output.mid')
    print("MIDI file created successfully.")  # Debug statement

# Ensure model and network_input are properly initialized
# model = ...  # Your trained model
# network_input = ...  # Your network input data

# Generate and save music
try:
    prediction_output = generate_music(model, network_input, int_to_note)
    create_midi(prediction_output)
except Exception as e:
    print(f"An error occurred: {e}")


Generated Music: ['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4'

In [3]:
# Import necessary modules from music21
from music21 import stream, note, midi

# Create a new stream to hold the melody
melody = stream.Stream()

# Define a sequence of notes for the melody
notes = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'B4', 'A4', 'G4', 'F4', 'E4', 'D4', 'C4']

# Add each note to the melody stream
for n in notes:
    melody.append(note.Note(n))

# Convert the melody stream to a MIDI file
mf = midi.translate.music21ObjectToMidiFile(melody)

# Open a new file to write the MIDI data
mf.open('longer_melody.mid', 'wb')

# Write the MIDI data to the file
mf.write()

# Close the file to ensure all data is saved
mf.close()
